In [6]:


import os
import pandas as pd
from google import genai
from google.colab import userdata


os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [7]:

try:
    client = genai.Client()
    print("Gemini client initialized successfully.")
except Exception as e:
    print(f"Error initializing client: {e}. Check your GEMINI_API_KEY.")


MODEL_NAME = "gemini-2.5-flash"

Gemini client initialized successfully.


In [8]:
def verify_diagnosis(symptoms, actual_diagnosis):
    """
    Sends the patient's symptoms and the doctor's diagnosis to the LLM for verification.
    """
    prompt = f"""
    You are a highly skilled medical AI assistant, specializing in diagnostic verification.
    Patient Symptoms (Input): '{symptoms}'.
    Doctor's Diagnosis (Output): '{actual_diagnosis}'.

    TASK:
    1. ASSESS if the symptoms provided highly suggest the recorded diagnosis.
    2. If the diagnosis is likely incorrect, provide the single most likely alternative diagnosis.
    3. STRICTLY output your response as a single, valid JSON object with the following keys:
       - "is_correct": (True or False)
       - "reasoning": (Detailed justification for your assessment)
       - "suggested_diagnosis": (The original diagnosis if correct, or the alternative if incorrect)
    """

    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt,
            config={"response_mime_type": "application/json"}
        )
        return response.text
    except Exception as e:
        return f'{{"is_correct": false, "reasoning": "API Error: {e}", "suggested_diagnosis": "N/A"}}'

In [9]:

df = pd.read_csv('/content/only_false_rows.csv')


df['llm_json_result'] = df.apply(
    lambda row: verify_diagnosis(row['input'], row['output']),
    axis=1
)


def safe_parse_json(json_string):
    try:
        return pd.read_json(json_string, typ='series')
    except ValueError:
        return pd.Series({'is_correct': None, 'reasoning': 'JSON Parsing Failed/LLM Error', 'suggested_diagnosis': 'Error'})


df_results = df['llm_json_result'].apply(safe_parse_json)
df = pd.concat([df.drop('llm_json_result', axis=1), df_results], axis=1)

print("\n--- Final Results DataFrame ---")
print(df)

/tmp/ipython-input-1368646839.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_string, typ='series')
/tmp/ipython-input-1368646839.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_string, typ='series')
/tmp/ipython-input-1368646839.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_string, typ='series')
/tmp/ipython-input-1368646839.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_st


--- Final Results DataFrame ---
                                                 input  \
0       hi doctor i think i might have turner syndrome   
1    doctor i am experiencing some problems during ...   
2    doctor i was diagnosed with congenital rubella...   
3    doctor my baby has been very fussy and irritab...   
4    doctor i have been experiencing severe stomach...   
..                                                 ...   
255  doctor i have been experiencing dizziness diff...   
256  doctor ive been having heartburn fluid retenti...   
257  hi doctor ive been experiencing sharp abdomina...   
258  doctor ive been experiencing chills sharp abdo...   
259  doctor i have been feeling a weakness in my lo...   

                                                output  similarity_score  \
0    okay in that case you will need to undergo a c...          0.299162   
1    i see in order to diagnose any possible compli...          0.248066   
2             you need to undergo a cornea

In [18]:
output_filename = 'diagnosis_verification_results.csv'
df.to_csv(output_filename, index=False, encoding='utf-8')
print(f"\n--- Results saved successfully ---")
print(f"File saved to: {output_filename}")
from google.colab import files
files.download(output_filename)


--- Results saved successfully ---
File saved to: diagnosis_verification_results.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>